### Imports

In [1]:
import itertools
import random
from typing import Any, Dict, List, Tuple

import numpy as np
import openml
import pandas as pd
from pandas import DataFrame
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor

# from evaluate_pipeline import evaluate_pipeline
# from load_dataset import load_dataset_from_id
# from perform_optimazaion_of_pipeline_with_random_search import (
#     perform_optimazaion_of_pipeline_with_random_search,
# )
# from split_dataset import split_dataset

### Utill functions

In [2]:
def optimize_pipeline_over_params_combinations(
    pipeline: Pipeline,
    parameters_grid: List[dict],
    X: DataFrame,
    y: DataFrame,
    X_val: DataFrame,
    y_val: DataFrame,
) -> Pipeline:
    # thats the teta^(j)*
    best_score = float("-inf")
    best_params = None

    for params in parameters_grid:
        # Update the pipeline parameters
        pipeline_params = {f"{key}": value for key, value in params.items()}
        pipeline.set_params(**pipeline_params)

        pipeline.fit(X, y)
        score = pipeline.score(X_val, y_val)

        if score > best_score:
            best_score = score
            best_params = pipeline_params

    pipeline.set_params(**best_params)
    return pipeline

In [3]:
def split_dataset(
    data: pd.DataFrame, class_: str
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    X: pd.DataFrame = data.drop(labels=class_, axis=1)
    y: pd.DataFrame = data[class_]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_train, y_test

In [4]:
def load_dataset_from_id(id: int) -> DataFrame:
    return openml.datasets.get_dataset(dataset_id=id).get_data(
        dataset_format="dataframe"
    )[0]

In [5]:
def calculate_mse(model: Pipeline, X_test: pd.DataFrame, y_test: pd.Series):
    # Ensure X_test and y_test are the correct types
    if not isinstance(X_test, (pd.DataFrame, np.ndarray)):
        raise ValueError("X_test must be a pandas DataFrame or numpy array")
    if not isinstance(y_test, (pd.Series, np.ndarray)):
        raise ValueError("y_test must be a pandas Series or numpy array")

    # Generating predictions and calculating MSE
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")

In [68]:


def evaluate_pipeline(
    pipeline: Pipeline,
    X: pd.DataFrame,
    y: pd.Series,
    X_val: pd.DataFrame,
    y_val: pd.Series,
):
    if not isinstance(X_val, (pd.DataFrame, np.ndarray)):
        raise ValueError("X_test must be a pandas DataFrame or numpy array")
    if not isinstance(y_val, (pd.Series, np.ndarray)):
        raise ValueError("y_test must be a pandas Series or numpy array")

    pipeline.fit(X, y)

    test_score = pipeline.score(X_val, y_val)
    train_score = pipeline.score(X, y)
    print("Parameter set: " + str(pipeline.named_steps["model"]))
    print("Test score R^2: " + str(test_score))
    print("Train score R^2: " + str(train_score))
    calculate_mse(pipeline, X_val, y_val)

### Load datasets

In [7]:
fish_market_dataset_id = (
    43308  # https://www.openml.org/search?type=data&id=43308&sort=runs&status=active
)
liver_disorders_dataset_id = (
    8  # https://www.openml.org/search?type=data&id=8&sort=runs&status=active
)
diabetes_dataset_id = (
    44223  # https://www.openml.org/search?type=data&id=44223&sort=runs&status=active
)

lisbona_house_prices_dataset_id = (
    43660  # https://www.openml.org/search?type=data&id=43660&sort=runs&status=active
)


fish_market_dataset: DataFrame = load_dataset_from_id(id=fish_market_dataset_id)
fish_market_regression_class = "Weight"

liver_disorders_dataset: DataFrame = load_dataset_from_id(id=liver_disorders_dataset_id)
liver_disorders_regression_class = "drinks"
diabetes_dataset: DataFrame = load_dataset_from_id(id=diabetes_dataset_id)
diabetes_regression_class = "class"

lisbona_house_prices_dataset: DataFrame = load_dataset_from_id(
    id=lisbona_house_prices_dataset_id
)
lisbona_house_prices_regression_class = "Price"

c:\JAN_CICHOMSKI\STUDIA\STUDIA_SEMESTR_7_2023_ZIMA\auto_ml\homeworks\homework_1\AutoML_HM1\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\JAN_CICHOMSKI\STUDIA\STUDIA_SEMESTR_7_2023_ZIMA\auto_ml\homeworks\homework_1\AutoML_HM1\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` 

### Split data into train and test sets

In [8]:
(
    X_train_fish_market,
    X_test_fish_market,
    y_train_fish_market,
    y_test_fish_market,
) = split_dataset(data=fish_market_dataset, class_=fish_market_regression_class)

(
    X_train_liver_disorders,
    X_test_liver_disorders,
    y_train_liver_disorders,
    y_test_liver_disorders,
) = split_dataset(data=liver_disorders_dataset, class_=liver_disorders_regression_class)

X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes = split_dataset(
    diabetes_dataset, diabetes_regression_class
)

(
    X_train_lisbona_house_prices,
    X_test_lisbona_house_prices,
    y_train_lisbona_house_prices,
    y_test_lisbona_house_prices,
) = split_dataset(lisbona_house_prices_dataset, lisbona_house_prices_regression_class)

### TODO: visualize the data

### Create generic column transformer

In [42]:
def get_column_transformer() -> ColumnTransformer:
    num_pipeline = Pipeline(
        steps=[("impute", SimpleImputer(strategy="mean")), ("scale", MinMaxScaler())]
    )
    cat_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="most_frequent")),
            ("one-hot", OneHotEncoder(handle_unknown="ignore", sparse=False)),
        ]
    )

    col_trans = ColumnTransformer(
        transformers=[
            (
                "num_pipeline",
                num_pipeline,
                make_column_selector(dtype_include=np.number),
            ),
            ("cat_pipeline", cat_pipeline, make_column_selector(dtype_include=object)),
        ],
        remainder="drop",
        n_jobs=-1,
    )
    return col_trans

In [43]:
col_trans: ColumnTransformer = get_column_transformer()

### Create models TODO: create also otehr pipelines

In [10]:
elastic_net = ElasticNet()
knn = KNeighborsRegressor()

### Create pipelines

#### 1. Decision Tree

In [45]:
def get_decision_tree_pipeline() -> Pipeline:
    decision_tree = DecisionTreeRegressor()
    decision_tree_pipeline = Pipeline(
        steps=[("column_transformer", col_trans), ("model", decision_tree)]
    )
    return decision_tree_pipeline

In [46]:
decision_tree_pipeline: Pipeline = get_decision_tree_pipeline()

#### Select parameters

In [56]:
# parameters space
ccp_alpha_values = [i * 0.1 for i in range(11)]
max_depth_values = range(1, 31, 1)
min_samples_split_values = range(2, 61, 1)
min_samples_leaf_values = range(1, 61, 1)

all_combinations = list(
    itertools.product(
        ccp_alpha_values,
        max_depth_values,
        min_samples_split_values,
        min_samples_leaf_values,
    )
)
selected_combinations: List[Tuple[float, int, int, int]] = random.sample(
    all_combinations, 100
)
parameter_names = [
    "model__ccp_alpha",
    "model__max_depth",
    "model__min_samples_split",
    "model__min_samples_leaf",
]

parameters_grid = [
    dict(zip(parameter_names, combination)) for combination in selected_combinations
]

#### Train and evaluate the model on fish market dataset

In [48]:
# ecision_tree_pipeline = optimize_pipeline_over_params_combinations(
#     pipeline=decision_tree_pipeline,
#     parameters_grid=parameters_grid,
#     X=X_train_fish_market,
#     y=y_train_fish_market,
#     X_val=X_test_fish_market,
#     y_val=y_test_fish_market,
# )

# evaluate_pipeline(
#     pipeline=decision_tree_pipeline,
#     X=X_train_fish_market,
#     y=y_train_fish_market,
#     X_val=X_test_fish_market,
#     y_val=y_test_fish_market,
# )

#### Train the model on liver disorders dataset

In [49]:
# decision_tree_pipeline = optimize_pipeline_over_params_combinations(
#     pipeline=decision_tree_pipeline,
#     parameters_grid=parameters_grid,
#     X=X_train_lisbona_house_prices,
#     y=y_train_lisbona_house_prices,
#     X_val=X_test_lisbona_house_prices,
#     y_val=y_test_lisbona_house_prices,
# )

# evaluate_pipeline(
#     pipeline=decision_tree_pipeline,
#     X=X_train_lisbona_house_prices,
#     y=y_train_lisbona_house_prices,
#     X_val=X_test_lisbona_house_prices,
#     y_val=y_test_lisbona_house_prices,
# )

In [50]:
def evaluate_model_performance(model, X, y):
    # Implement the model training and evaluation here
    # Return the performance metric (e.g., accuracy, MSE, etc.)
    model.fit(X, y)
    return model.score(X, y)

In [51]:
def experiment(datasets, model, config):
    performances = []
    for X, y in datasets:
        model.set_params(**config)
        performance = evaluate_model_performance(model, X, y)
        performances.append(performance)
    return performances

In [60]:
def find_optimal_configuration(config_space, datasets, model, summary_func):
    best_config = None
    best_summary_score = float("0")

    for config in config_space:
        performances = experiment(datasets, model, config)
        summary_score = summary_func(performances)

        if summary_score > best_summary_score:
            best_summary_score = summary_score
            best_config = config

    return best_config

In [61]:
datasets = [
    (X_train_fish_market, y_train_fish_market),
    (X_train_lisbona_house_prices, y_train_lisbona_house_prices),
]


optimal_config = find_optimal_configuration(
    config_space=parameters_grid,
    datasets=datasets,
    model=decision_tree_pipeline,
    summary_func=np.mean,  # Or np.median for a more robust approach
)

In [64]:
print(optimal_config)

{'model__ccp_alpha': 0.0, 'model__max_depth': 18, 'model__min_samples_split': 3, 'model__min_samples_leaf': 2}


In [65]:
decision_tree_pipeline = get_decision_tree_pipeline()
decision_tree_pipeline.set_params(**optimal_config)

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000261F15AACA0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000261F15F87F0>)])),
                ('model',
                 DecisionTreeRegressor(max_depth=18, min_samples_leaf=2,
                                       min_samples_split=3))])

In [69]:
evaluate_pipeline(
    pipeline=decision_tree_pipeline,
    X=X_train_fish_market,
    y=y_train_fish_market,
    X_val=X_test_fish_market,
    y_val=y_test_fish_market,
)

evaluate_pipeline(
    pipeline=decision_tree_pipeline,
    X=X_train_lisbona_house_prices,
    y=y_train_lisbona_house_prices,
    X_val=X_test_lisbona_house_prices,
    y_val=y_test_lisbona_house_prices,
)

Parameter set: DecisionTreeRegressor(max_depth=18, min_samples_leaf=2, min_samples_split=3)
Test score R^2: 0.9500345734380512
Train score R^2: 0.9958440148292846
Mean Squared Error: 7107.046640625
Parameter set: DecisionTreeRegressor(max_depth=18, min_samples_leaf=2, min_samples_split=3)
Test score R^2: 0.6769996507433077
Train score R^2: 0.9294768799395877
Mean Squared Error: 18175128262.864445
